In [2]:
## Use DASH to build user GUI
# %pip install dash
# %pip install nbformat

from dash import Dash, html, dcc, callback, Output, Input, ctx, dash_table
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

from presets import setmedium
import calculateRayleighIntegral
import settransducer


In [ ]:
app = Dash('tempCompliance4HIFU')

# df_pressure2D = pd.read_csv(r"C:\Users\geral\OneDrive\Documents\GitHub\TempCompliance4HIFU\src\matlab_data_1MHz_2cm_5cm.csv")  # sample_data header=none
# df_pressure2D = np.rot90(df_pressure2D)
# z_axis = (np.linspace(0,0.1,100)) * 1e3
# r_axis = (np.linspace(-0.02,0.02,101)) * 1e3

app.layout = [

    # Title
    html.H1(children='Temperature Compliance for HIFU Transducers',style={'textAlign':'center'}), 


    # TOP Section - Button and Messages
    html.Div([
        html.Button('CALCULATE PRESSURE AND TEMPERATURE',id='button',n_clicks=0),
        html.Div(id='message_Display',style={'textAlign':'left'}),
    ],style={'width': '50%','float': 'right','display': 'inline-block','padding': '10px 5px'}),
    
    
    # RIGHT Section - DISPLAYS
    html.Div([
        html.H2('Results', style ={'textAlign':'center'}), 
            dcc.Dropdown(['Pressure','Intensity'],'Intensity',id='DROP_field2D'),
            dcc.Graph(id='GRAPH_field2D',hoverData={'points': [{'customdata': 'Pressure'}]}),
            dcc.Graph(id='GRAPH_time1D'),
    ],style={'width': '50%', 'float': 'right', 'display': 'inline-block'}),


    # SPACE
    html.Div([html.H3(' ',style ={'textAlign':'center'})],style={'width': '3%', 'float': 'right', 'display': 'inline-block'}),


    # MIDDLE Section - INPUTS
    html.Div([     
            
        # Trandsucer Section (Top)
        html.H3('Transducer', style ={'textAlign':'center'}), 
        html.P('Frequency [MHz]', style ={'textAlign':'left'}),
            dcc.Input(id="Frequency", type='number', placeholder='Input Frequency',value=1e6),
        html.P('Radius [mm]', style ={'textAlign':'left'}),
            dcc.Input(id="Radius", type='number', placeholder='Input Radius',value=0.02),
        html.P('Focus Distance [mm]', style ={'textAlign':'left'}),
            dcc.Input(id="Focus", type='number', placeholder='Input Focus Distance',value=0.05),
        html.P('Initial Pressure [mm]', style ={'textAlign':'left'}),
            dcc.Input(id="InitPress", type='number', placeholder='Input Initial Pressure',value=1),


        # Heating Section (Middle)
        html.H3('Heating', style ={'textAlign':'center'}), 
        html.P('Time Heating [s]', style ={'textAlign':'left'}),
            dcc.Input(id="HeatTime", type='number', placeholder='Input Heat Time'),
        html.P('Time Cooling [s]', style ={'textAlign':'left'}),
            dcc.Input(id="CoolTime", type='number', placeholder='Input Cool Time'),
        html.P('Duty Cycle [Percentage]', style ={'textAlign':'left'}),
            dcc.Input(id="DutyCycle", type='number', placeholder='Input Duty Cycle'),


        # Plotting Section
        html.H3('Plotting Parameters', style ={'textAlign':'center'}), 
        html.P('Number of Axial Steps', style ={'textAlign':'left'}),
            dcc.Input(id="numZ", type='number', placeholder='Input # Axial Steps',value=100),
        html.P('Number of Radial Steps', style ={'textAlign':'left'}),
            dcc.Input(id="numR", type='number', placeholder='Input # Radial Steps',value=100),
        html.P('Time Step', style ={'textAlign':'left'}),
            dcc.Input(id="numTime", type='number', placeholder='Input Time Step'),

    ], style={'width': '15%', 'float': 'right', 'display': 'inline-block'}),


    # SPACE
    html.Div([html.H3(' ',style ={'textAlign':'center'})],style={'width': '3%', 'float': 'right', 'display': 'inline-block'}),


    # LEFT Section - Medium 
    html.Div([  
        html.H3('Medium', style ={'textAlign':'center'}), 
        html.P('Presets', style ={'textAlign':'left'}), 
            dcc.Dropdown(['Custom','Water','Glycerol','Egg White','Castor Oil'],'Water',id='DROP_medium'),
        html.P('Speed of Sound [m/s]', style ={'textAlign':'left'}), 
            dcc.Input(id="Speed", type='number', placeholder='Input Speed of Sound'),
        html.P('Density [kg/m^3]', style ={'textAlign':'left'}),
            dcc.Input(id="Density", type='number', placeholder='Input Density'),
        html.P('Absorption Coeffient [Np/(m*MHz^2)]', style ={'textAlign':'left'}),
            dcc.Input(id="AbsCoeff", type='number', placeholder='Input Absorption Coefficient'),
        html.P('Specific Heat Capacity [J/(kg*K)]', style ={'textAlign':'left'}),
            dcc.Input(id="SpecHeatCap", type='number', placeholder='Input Specific Heat Capacity'),
        html.P('Thermal Diffusivity [(m^2)/s]', style ={'textAlign':'left'}),
            dcc.Input(id="ThermDiff", type='number', placeholder='Input Thermal Diffusivity'),  
        
        html.P('Call Data Files', style ={'textAlign':'left'}),
            dcc.Input(id="dataFrame",type='text',value="matlab_data_1MHz_2cm_5cm.csv"),
            dcc.Input(id="ZAXE",type='text',value="z_axis_plchldr.csv"),
            dcc.Input(id="RAXE",type='text',value="r_axis_plchldr.csv"),


    ], style={'width': '15%', 'float': 'right', 'display': 'inline-block'}),

]

##### ##### Callbacks ##### ##### 


# Update 2D Figure (Pressure or Intensity)
@callback(
    Output('GRAPH_field2D', 'figure'),
    Input('DROP_field2D','value'),
    Input('dataFrame','value'),
    Input('ZAXE','value'),
    Input('RAXE','value'),
)
def update2Dfigure(DROP_field2D,filename1,filename2,filename3):
    # Update the 2D Figure Depending on Pressure or Intensity Selection
    directory = os.getcwd()     
    df_pressure2D = np.rot90(np.array(pd.read_csv((directory + "\\" +  filename1))))
    z_axis = np.array(pd.read_csv((directory + "\\" +  filename2),header=None))[0]
    r_axis = np.array(pd.read_csv((directory + "\\" +  filename3),header=None))[0]

    match DROP_field2D:
        case 'Intensity':
            display_array = pow(df_pressure2D,2)
            colorbarLabel = "Intensity [p/p0]^2"
            titleLabel = "Intensity Map"
        case 'Pressure':
            display_array = df_pressure2D
            colorbarLabel = "Pressure [p/p0]"
            titleLabel = "Pressure Map"

    # MatPlotLib Figure Structure
    fig2D = px.imshow(
        display_array, 
        x = z_axis, 
        y = r_axis,
        labels={"x":"Z-Axis [mm]", "y":"R-Axis [mm]", "color":colorbarLabel},
        title=titleLabel,
        aspect='auto', 
        origin='lower',
        color_continuous_scale='jet', 
    )
    return fig2D


# Set Medium Properties Callback
@callback(
    Output('Speed','value'),
    Output('Density','value'),
    Output('AbsCoeff','value'),
    Output('SpecHeatCap','value'),
    Output('ThermDiff','value'),
    Input('DROP_medium','value'),
    Input('Speed','value'),
    Input('Density','value'),
    Input('AbsCoeff','value'),
    Input('SpecHeatCap','value'),
    Input('ThermDiff','value'),
)
def getMedium(DROP_medium, Speed, Density, AbsCoeff, SpecHeatCap, ThermDiff):
    mediumUserIn = collectMediumUserInputs(Speed, Density, AbsCoeff, SpecHeatCap, ThermDiff)
    mediumProp = setmedium.setMedium(DROP_medium, mediumUserIn)
    return mediumProp['speed'], mediumProp['density'], mediumProp['absCoeff'], mediumProp['specHeatCap'], mediumProp['thermDiff']

def collectMediumUserInputs(Speed, Density, AbsCoeff, SpecHeatCap, ThermDiff):
    if Speed == None: Speed = 0
    if Density == None: Density = 0
    if AbsCoeff == None: AbsCoeff = 0
    if SpecHeatCap == None: SpecHeatCap = 0
    if ThermDiff == None: ThermDiff = 0
    userInputs = dict(speed = Speed, density = Density, absCoeff = AbsCoeff, specHeatCap = SpecHeatCap, thermDiff = ThermDiff)
    return userInputs


# Calculate Pressure and Temperature Field (When Button is Pressed)
@callback(
    Output('message_Display', 'children'),
    Output('dataFrame','value'),
    Output('ZAXE','value'),
    Output('RAXE','value'),

    Input('button','n_clicks'),

    Input('Frequency','value'),
    Input('Radius','value'),
    Input('Focus','value'),
    Input('InitPress','value'),

    Input('DROP_medium','value'),
    Input('Speed','value'),
    Input('Density','value'),
    Input('AbsCoeff','value'),
    Input('SpecHeatCap','value'),
    Input('ThermDiff','value'),

    Input('numZ','value'),
    Input('numR','value'),

    # Input('HeatTime','value'),
    # Input('CoolTime','value'),
    # Input('DutyCycle','value'),

    Input('dataFrame','value'),
    Input('ZAXE','value'),
    Input('RAXE','value'),

)
def calculate2DField(buttonClicks,Frequency, Radius, Focus, InitPress, 
                     DROP_medium, Speed, Density, AbsCoeff, SpecHeatCap, ThermDiff,
                     numZ, numR, 
                     filename1, filename2, filename3):
    # When the Button is Pressed
    if 'button' == ctx.triggered_id:
        msg = "Calculating... [Estimated Time Completion 3.5 Minutes]"
        
        # Set Input Parameters
        trans = settransducer.setTransducer(Frequency,Radius,Focus,InitPress) # freq, radius, focus, init pressure
        mediumUserIn = collectMediumUserInputs(Speed, Density, AbsCoeff, SpecHeatCap, ThermDiff)
        medium = setmedium.setMedium(DROP_medium, mediumUserIn)
        field = dict(numAxialStep = numZ, numRadialStep = numR)

        # Rayleigh Integral
        df_pressure2D, z_axis, r_axis, iscomplete = calculateRayleighIntegral.generateField(trans,medium,field)
        
        df_pressure2D = pd.DataFrame(df_pressure2D)
        z_axis = pd.DataFrame(z_axis)
        r_axis = pd.DataFrame(r_axis)

        filename1="df_pressure2D.csv"
        df_pressure2D.to_csv(filename1)
        filename2="z_axis.csv"
        z_axis.to_csv(filename2)
        filename3="r_axis.csv"
        r_axis.to_csv(filename3)

        if iscomplete == 0:
            msg = "ERROR: RAYLEIGH INTEGRAL INCOMPLETE"
            return html.Div(msg), filename1, filename2, filename3
        
        msg = 'Complete'
        return html.Div(msg), filename1, filename2, filename3

    # When Button is Not Pressed
    else:
        if buttonClicks == 0:
            msg = "Program Loaded In Browser. Awaiting Inputs"
            return html.Div(msg), filename1, filename2, filename3
        else:
            msg = "_"
            return html.Div(msg), filename1, filename2, filename3






# Execute App
if __name__ == '__main__':
    app.run(debug=True)


1.0 %
2.0 %
3.0 %
4.0 %
5.0 %
6.0 %
7.0 %
8.0 %
9.0 %
10.0 %
11.0 %
12.0 %
13.0 %
14.0 %
15.0 %
16.0 %
17.0 %
18.0 %
19.0 %
20.0 %
21.0 %
22.0 %
23.0 %
24.0 %
25.0 %
26.0 %
27.0 %
28.0 %
29.0 %
30.0 %
31.0 %
32.0 %
33.0 %
34.0 %
35.0 %
36.0 %
37.0 %
38.0 %
39.0 %
40.0 %
41.0 %
42.0 %
43.0 %
44.0 %
45.0 %
46.0 %
47.0 %
48.0 %
49.0 %
50.0 %
50.0 %
51.0 %
52.0 %
53.0 %
54.0 %
55.0 %
56.0 %
57.0 %
58.0 %
59.0 %
60.0 %
61.0 %
62.0 %
63.0 %
64.0 %
65.0 %
66.0 %
67.0 %
68.0 %
69.0 %
70.0 %
71.0 %
72.0 %
73.0 %
74.0 %
75.0 %
76.0 %
77.0 %
78.0 %
79.0 %
80.0 %
81.0 %
82.0 %
83.0 %
84.0 %
85.0 %
86.0 %
87.0 %
88.0 %
89.0 %
90.0 %
91.0 %
92.0 %
93.0 %
94.0 %
95.0 %
96.0 %
97.0 %
98.0 %
99.0 %
100.0 %
Rayleigh Integral Complete
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[34], line 134, in update2Dfigure(
    DROP_field2D='Intensity',
    filename1='df_pressure2D.csv',
    filename2='

In [29]:
filename2 = "z_axis_plchldr.csv"
directory = os.getcwd()     


z_axis = np.array(pd.read_csv((directory + "\\" +  filename2),header=None))[0]
print(z_axis)
type(z_axis)

[  0.           1.01010101   2.02020202   3.03030303   4.04040404
   5.05050505   6.06060606   7.07070707   8.08080808   9.09090909
  10.1010101   11.11111111  12.12121212  13.13131313  14.14141414
  15.15151515  16.16161616  17.17171717  18.18181818  19.19191919
  20.2020202   21.21212121  22.22222222  23.23232323  24.24242424
  25.25252525  26.26262626  27.27272727  28.28282828  29.29292929
  30.3030303   31.31313131  32.32323232  33.33333333  34.34343434
  35.35353535  36.36363636  37.37373737  38.38383838  39.39393939
  40.4040404   41.41414141  42.42424242  43.43434343  44.44444444
  45.45454545  46.46464646  47.47474747  48.48484848  49.49494949
  50.50505051  51.51515152  52.52525253  53.53535354  54.54545455
  55.55555556  56.56565657  57.57575758  58.58585859  59.5959596
  60.60606061  61.61616162  62.62626263  63.63636364  64.64646465
  65.65656566  66.66666667  67.67676768  68.68686869  69.6969697
  70.70707071  71.71717172  72.72727273  73.73737374  74.74747475
  75.7575757

numpy.ndarray

In [33]:
filename1 = "matlab_data_1MHz_2cm_5cm.csv"
filename3 = "r_axis_plchldr.csv"


df_pressure2D = np.array(pd.read_csv((directory + "\\" +  filename1)))
z_axis = np.array(pd.read_csv((directory + "\\" +  filename2),header=None))[0]
r_axis = np.array(pd.read_csv((directory + "\\" +  filename3),header=None))[0]

print(len(df_pressure2D))
print(len(z_axis))
print(len(r_axis))

100
100
101


In [3]:
df_pressure2D = pd.read_csv(r"C:\Users\geral\OneDrive\Documents\GitHub\TempCompliance4HIFU\src\matlab_data_1MHz_2cm_5cm.csv",header=None)  # sample_data

df_pressure2D.transpose().to_dict()

{0: {0: 0.95201881241458,
  1: 1.11238354541875,
  2: 1.13800500810308,
  3: 1.00813647909543,
  4: 0.82966202287875,
  5: 0.843122321398373,
  6: 1.05947037603107,
  7: 1.09779905360773,
  8: 0.892166655638951,
  9: 0.865028385591683,
  10: 1.03196451889168,
  11: 1.01524560576967,
  12: 0.922847870283726,
  13: 1.0476030161761,
  14: 1.01157885826008,
  15: 0.852140940406974,
  16: 0.985020449130516,
  17: 1.11872861278902,
  18: 0.955486472129014,
  19: 0.924796633450443,
  20: 1.01203452546107,
  21: 0.932890954395149,
  22: 0.905695625970314,
  23: 0.995500457408041,
  24: 0.890172836084774,
  25: 4.07084977209162,
  26: 1.20061875634272,
  27: 0.995167737281992,
  28: 0.867341666222601,
  29: 0.905679888467635,
  30: 0.94579194603975,
  31: 0.959134591576393,
  32: 1.09184726239417,
  33: 1.08030817734062,
  34: 0.866211393907174,
  35: 0.932005540357737,
  36: 1.18998686324239,
  37: 1.07521515205502,
  38: 0.835415816541374,
  39: 0.986783203527983,
  40: 1.19616590838424,
  41

In [13]:
filename1 = "matlab_data_1MHz_2cm_5cm.csv"

directory = os.getcwd()     

df_pressure2D = pd.read_csv((directory + "\\" +  filename1))